# Master Thesis 2nd Version of a model on spontaneous speech

**Author**: Karin Thommen

**Date**: May 2023 


---

**Content of the Notebook**:  Fine-tuning and Training of OpenAi Whisper ASR Model

---

## Step 1: Import and Setup

In [ ]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install audio-metadata
!pip install "dill<0.3.5"
!pip install git-lfs
!pip install pydub

In [ ]:
import pandas as pd
import os
import transformers

from datasets.fingerprint import Hasher
import pickle
import dill

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json

import IPython.display as ipd
import numpy as np
import random

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import audio_metadata

from datasets import load_dataset, Audio, load_metric, load_from_disk, DatasetDict, list_datasets
from datasets import Dataset, Sequence

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import WhisperTokenizer
from transformers import WhisperTokenizerFast
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from huggingface_hub import notebook_login

from google.colab import drive

from moviepy.editor import concatenate_audioclips, AudioFileClip


In [ ]:
transformers.__version__

'4.28.0'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Step 2: Load Data

In [ ]:
# Build connection to data folder on GDrive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# login to huggingface account for data
notebook_login()

In [ ]:
# load dataset from huggingface (after uploading it via local machine to huggingface)
dataset = load_dataset("karinthommen/schawinski_V2")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/645 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/647 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/karinthommen___parquet/karinthommen--schawinski_V2-6a970eecd5fc90fe/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# check if data loading worked
dataset["train"][0]

{'audio': {'path': 'Badran_Schawinski_13-05-2013_SPK0-Badran_Schawinski_13-05-2013-0001.wav',
  'array': array([ 0.00010681,  0.00018311,  0.00045776, ...,  0.00128174,
         -0.00854492, -0.01789856]),
  'sampling_rate': 44100},
 'transcription': '[music]',
 'duration': 26.31}

In [ ]:
dataset.shape

{'train': (3544, 3), 'validation': (645, 3), 'test': (647, 3)}

In [ ]:
def preprocess(batch):
  batch["transcription"] = re.sub('\[music\]', '', batch["transcription"])
  batch["transcription"] = re.sub('\[breath_mouth_noise\]', '', batch["transcription"])
  batch["transcription"] = re.sub('\[laughter\]', '', batch["transcription"])
  batch["transcription"] = re.sub('\[speech-in-speech\]', '', batch["transcription"])
  batch["transcription"] = re.sub(r"\\", '', batch["transcription"])
  batch["transcription"] = re.sub('\*', '', batch["transcription"])
  return batch

In [ ]:
dataset = dataset.filter(lambda example: not example["transcription"].startswith("[speech-in-speech]"))

Filter:   0%|          | 0/3544 [00:00<?, ? examples/s]

Filter:   0%|          | 0/645 [00:00<?, ? examples/s]

Filter:   0%|          | 0/647 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.map(preprocess, num_proc=1)

Map:   0%|          | 0/1962 [00:00<?, ? examples/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/393 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.filter(lambda example: example["transcription"]!="")

Filter:   0%|          | 0/1962 [00:00<?, ? examples/s]

Filter:   0%|          | 0/303 [00:00<?, ? examples/s]

Filter:   0%|          | 0/393 [00:00<?, ? examples/s]

In [ ]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\$]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    batch["transcription"] = batch["transcription"].strip()
    return batch

In [ ]:
dataset = dataset.map(remove_special_characters, num_proc=1)

Map:   0%|          | 0/1946 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Map:   0%|          | 0/393 [00:00<?, ? examples/s]

In [ ]:
# check if preprocessing loading worked
dataset["train"][0]

{'audio': {'path': 'Badran_Schawinski_13-05-2013_SPK0-Badran_Schawinski_13-05-2013-0002.wav',
  'array': array([-0.01348877, -0.01480103, -0.03569031, ...,  0.00212097,
          0.00344849,  0.00289917]),
  'sampling_rate': 44100},
 'transcription': 'da isch di eerschti taakschou vo de wuche froit m/ mi dass si au hüt aabig debii sind bi miir isch d schagglin badraan ässphee nazionaalräätin',
 'duration': 6.39}

In [ ]:
dataset.shape

{'train': (1946, 3), 'validation': (302, 3), 'test': (393, 3)}

In [ ]:
# load tokenizer form Whisper Tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

In [ ]:
# load feature extractor 
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
# downsample dataset to a sampling rate of 16kHz for the model 
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# Check if audio loading worked with a random audio and sentence 
rand_int = random.randint(0, len(dataset["train"])-1)
print(dataset["train"]["transcription"][rand_int])
ipd.Audio(data=dataset["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

[speechinnoise] und das fröög ich mich aso du häsch da riisigi hassienda


In [ ]:
# Check sentence, input array shape and sampling rate 
rand_int = random.randint(0, len(dataset["train"])-1)

print("Target text:", dataset["train"][rand_int]["transcription"])
print("Input array shape:", dataset["train"][rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset["train"][rand_int]["audio"]["sampling_rate"])

Target text: d schtatistik isch vil
Input array shape: (17280,)
Sampling rate: 16000


In [ ]:
# show sentence decoded with the special characters ( in the format that is needed by whisper )
input_str = dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 da isch di eerschti taakschou vo de wuche froit m/ mi dass si au hüt aabig debii sind bi miir isch d schagglin badraan ässphee nazionaalräätin
Decoded w/ special:    <|startoftranscript|><|transcribe|><|notimestamps|>da isch di eerschti taakschou vo de wuche froit m/ mi dass si au hüt aabig debii sind bi miir isch d schagglin badraan ässphee nazionaalräätin<|endoftext|>
Decoded w/out special: da isch di eerschti taakschou vo de wuche froit m/ mi dass si au hüt aabig debii sind bi miir isch d schagglin badraan ässphee nazionaalräätin
Are equal:             True


In [ ]:
def concat_audio(audio_paths, output_path): # concatenate audios to one audio 
    clips = [AudioFileClip(c) for c in audio_clip_paths]
    final_clip = concatenate_audioclips(clips)

dataset["test"]

Dataset({
    features: ['audio', 'transcription', 'duration'],
    num_rows: 393
})

In [ ]:
# load processor from Whisper Processor 
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, num_proc=2)

Map (num_proc=2):   0%|          | 0/1946 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/302 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/393 [00:00<?, ? examples/s]

In [ ]:
dataset.push_to_hub("karinthommen/schawinski-features-no-vocab", private=True)

## Fine-Tune & Train Model 

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./spontaneous-whisper-v2",  
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False, 
    hub_private_repo=True,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/karinthommen/spontaneous-whisper-v2 into local empty directory.


In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

  warnings.warn(

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.025600,1.096072,66.968034
2000,0.001400,1.229760,65.708750
3000,0.000400,1.289355,65.385857
4000,0.000300,1.310084,65.224411


TrainOutput(global_step=4000, training_loss=0.20291685020131991, metrics={'train_runtime': 24814.0168, 'train_samples_per_second': 2.579, 'train_steps_per_second': 0.161, 'total_flos': 1.841405722361856e+19, 'train_loss': 0.20291685020131991, 'epoch': 32.79})